# Вариант 2. Задание 2. SQL
## 2.1 Очень усердные ученики

**2.1.1 Условие**

Образовательные курсы состоят из различных уроков, каждый из которых состоит из нескольких маленьких заданий. Каждое такое маленькое задание называется "горошиной".

Назовём очень усердным учеником того пользователя, который хотя бы раз за текущий месяц правильно решил 20 горошин.

**2.1.2 Задача**

Дана таблица ***default.peas***:


|Название атрибута|	|Тип атрибута|	|Смысловое значение                      |
|:----------------|-|:-----------|--|:---------------------------------------|
|st_id            | |int         |  |ID ученика                              |
|timest           | |timestamp   |  |Время решения карточки                  |
|correct          | |bool        |  |Правильно ли решена горошина?           |
|subject          | |text        |  |Дисциплина, в которой находится горошина|


		


Необходимо написать *оптимальный запрос*, который даст информацию о количестве очень усердных студентов. *NB!* Под усердным студентом мы понимаем студента, который правильно решил 20 задач за текущий месяц.

In [61]:
import pandahouse as ph

In [62]:
connection_default = {'host': 'https://clickhouse.lab.karpov.courses',
                      'database': 'default',
                      'user': 'student', 
                      'password': 'dpo_python_2020'}

**посмотрим на данные:**

In [63]:
df_eda = '''
    SELECT
        st_id,
        toDateTime(timest),
        correct,
        subject
    FROM default.peas
    LIMIT 10
'''

ph.read_clickhouse(df, connection=connection_default)

,hard_working_studs
0,136


**посмотрим на уникальные даты:**

In [64]:
df_uniq = '''
    SELECT toDate(timest) as uniq_date 
    FROM default.peas
    GROUP BY toDate(timest)
    LIMIT 10
'''

ph.read_clickhouse(df_uniq, connection=connection_default)

,uniq_date
0,2021-10-30
1,2021-10-31


*в таблице всего две даты в октябре 2021, т.е в доп отборе дат нет необходимости*

In [65]:
df = '''
SELECT COUNT(*) as hard_working_studs
    FROM(
        SELECT st_id, SUM(correct) AS count_correct
        FROM default.peas
        GROUP BY st_id
        HAVING count_correct >= 20
        ) AS df
'''

ph.read_clickhouse(df_final, connection=connection_default)

,hard_working_studs
0,136


**количество очень усердных учеников - 136**

## 2.2 Оптимизация воронки

**2.2.1 Условие**

Образовательная платформа предлагает пройти студентам курсы по модели trial: студент может решить бесплатно лишь 30 горошин в день. Для неограниченного количества заданий в определенной дисциплине студенту необходимо приобрести полный доступ. Команда провела эксперимент, где был протестирован новый экран оплаты.

**2.2.2 Задача**

Дана таблицы: ***default.peas*** (см. выше), ***default.studs***:

|Название атрибута|	|Тип атрибута|	|Смысловое значение                      |
|:----------------|-|:-----------|--|:---------------------------------------|
|st_id            | |int         |  |ID ученика                              |
|test_grp         |	|text	     |  |Метка ученика в данном эксперименте     |

и ***default.final_project_check***:

|Название атрибута|	|Тип атрибута|	|Смысловое значение                       |
|:----------------|-|:-----------|--|:----------------------------------------|
|st_id            | |int         |  |ID ученика                               |
|sale_time        | |timestamp   |  |Время покупки                            |
|money            | |int         |  |	Цена, по которой приобрели данный курс|
|subject          | |text        |  |                                         |

	 	
Необходимо в одном запросе выгрузить следующую информацию о группах пользователей:

- ARPU 
- ARPAU 
- CR в покупку 
- СR активного пользователя в покупку 
- CR пользователя из активности по математике (subject = ’math’) в покупку курса по математике

ARPU считается относительно всех пользователей, попавших в группы.

*Активным* считается пользователь, за все время решивший *больше* 10 задач правильно в любых дисциплинах.

*Активным* по математике считается пользователь, за все время решивший *2 или больше* задач правильно по математике.

*Все данные находятся в табличном виде в ClickHouse*

In [66]:
ARPU = '''
SELECT
    test_grp,
    round(SUM(all_revenue/cnt_uniq_st), 2) AS ARPU
FROM (
    SELECT
        s.test_grp AS test_grp,
        SUM(f.money) AS all_revenue
    FROM default.final_project_check AS f
    LEFT JOIN default.studs AS s ON f.st_id=s.st_id
    GROUP BY test_grp
    ) AS all_revenue
LEFT JOIN (
    SELECT
        s.test_grp,
        uniqExact(p.st_id) AS cnt_uniq_st
    FROM default.peas AS p
    LEFT JOIN default.studs AS s ON p.st_id=s.st_id
    GROUP BY s.test_grp
    ) AS cnt_uniq_st ON all_revenue.test_grp=cnt_uniq_st.test_grp
GROUP BY test_grp
'''

ph.read_clickhouse(tack_2_ARPU, connection=connection_default)

,test_grp,ARPU
0,control,8393.94
1,pilot,23741.26


In [68]:
ARPAU = '''
SELECT
    test_grp,
    round(SUM(n.avg_money/q.st_id), 2) AS ARPAU
FROM (
    SELECT
        test_grp,
        COUNT(st_id) AS st_id
    FROM (
        SELECT
            ccm.st_id,
            ccm.cnt_correct_more10,
            s.test_grp
        FROM (
            SELECT
                st_id,
                SUM(correct) AS cnt_correct_more10
            FROM default.peas
            GROUP BY st_id
            HAVING cnt_correct_more10 > 10
            ) AS ccm
        LEFT JOIN default.studs AS s ON ccm.st_id=s.st_id) AS a
        GROUP BY test_grp
        ) AS q
    LEFT JOIN (
        SELECT
            test_grp,
            AVG(money) AS avg_money
        FROM (
            SELECT 
                f.st_id,
                f.money,
                s.test_grp
            FROM default.final_project_check AS f
            LEFT JOIN default.studs AS s on f.st_id=s.st_id)
            GROUP BY test_grp
            ) AS n on n.test_grp=q.test_grp
GROUP BY test_grp
'''
ph.read_clickhouse(ARPAU, connection=connection_default)

,test_grp,ARPAU
0,control,605.86
1,pilot,822.43


In [72]:
CR = '''
SELECT
    r.test_grp,
    round(SUM(buyers/all_st), 4) AS CR
FROM (
    SELECT
        s.test_grp,
        uniqExact(f.st_id) AS buyers
    FROM default.final_project_check AS f
    LEFT JOIN default.studs AS s ON f.st_id=s.st_id
    GROUP BY test_grp
    ) AS r
LEFT JOIN (
    SELECT
        s.test_grp,
        uniqExact(p.st_id) AS all_st
    FROM default.peas AS p
    LEFT JOIN default.studs AS s ON p.st_id=s.st_id
    GROUP BY test_grp
    ) AS w ON r.test_grp=w.test_grp
GROUP BY test_grp
'''

ph.read_clickhouse(CR, connection=connection_default)

,test_grp,CR
0,control,0.0909
1,pilot,0.2238


In [76]:
CR_active = '''
SELECT
    test_grp,
    round((y.buyers/t.st_id), 4) AS CR_active
FROM (
    SELECT
        test_grp,
        COUNT(st_id) AS st_id
    FROM (
        SELECT
            ccm.st_id,
            ccm.cnt_correct_more10,
            s.test_grp
        FROM (
            SELECT
                st_id,
                sum(correct) AS cnt_correct_more10
            FROM default.peas
            GROUP BY st_id
            HAVING cnt_correct_more10 > 10
            ) AS ccm
        LEFT JOIN default.studs AS s ON ccm.st_id=s.st_id) AS a
        group by test_grp) as t
    LEFT JOIN (
        SELECT
            s.test_grp,
            uniqExact(f.st_id) AS buyers
        FROM default.final_project_check AS f
        LEFT JOIN default.studs AS s ON f.st_id=s.st_id
        GROUP BY test_grp
        ) AS y ON t.test_grp=y.test_grp
'''
ph.read_clickhouse(CR_active, connection=connection_default)

,test_grp,CR_active
0,control,0.1181
1,pilot,0.3333


In [91]:
CR_math = '''
SELECT
    test_grp,
    round((cnt_buy_math/cnt_math), 4) AS CR_math
FROM (
    SELECT
        s.test_grp,
        count(u.st_id) AS cnt_math
    FROM (
        SELECT 
            st_id,
            SUM(correct) AS cnt_correct_2_and_more
        FROM default.peas
        WHERE subject = 'Math'
        GROUP BY st_id
        HAVING cnt_correct_2_and_more >= 2
        ) AS u 
    LEFT JOIN default.studs AS s ON u.st_id=s.st_id
    GROUP BY test_grp
    ) AS i
LEFT JOIN (
    SELECT
        test_grp,
        uniqExact(st_id) AS cnt_buy_math
    FROM (
        SELECT *
        FROM default.final_project_check AS f
        LEFT JOIN default.studs AS s ON f.st_id=s.st_id
        WHERE subject = 'Math'
        )
    GROUP BY test_grp) AS l ON l.test_grp=i.test_grp
'''
ph.read_clickhouse(CR_math, connection=connection_default)

,test_grp,CR_math
0,control,0.1633
1,pilot,0.3810


In [110]:
# объединяю в один запрос:

total = '''
WITH arpu AS (SELECT
    test_grp,
    round(SUM(all_revenue/cnt_uniq_st), 2) AS ARPU
FROM (
    SELECT
        s.test_grp AS test_grp,
        SUM(f.money) AS all_revenue
    FROM default.final_project_check AS f
    LEFT JOIN default.studs AS s ON f.st_id=s.st_id
    GROUP BY test_grp
    ) AS all_revenue
LEFT JOIN (
    SELECT
        s.test_grp,
        uniqExact(p.st_id) AS cnt_uniq_st
    FROM default.peas AS p
    LEFT JOIN default.studs AS s ON p.st_id=s.st_id
    GROUP BY s.test_grp
    ) AS cnt_uniq_st ON all_revenue.test_grp=cnt_uniq_st.test_grp
GROUP BY test_grp) ,


arpau AS (SELECT
    test_grp,
    round(SUM(n.avg_money/q.st_id), 2) AS ARPAU
FROM (
    SELECT
        test_grp,
        COUNT(st_id) AS st_id
    FROM (
        SELECT
            ccm.st_id,
            ccm.cnt_correct_more10,
            s.test_grp
        FROM (
            SELECT
                st_id,
                SUM(correct) AS cnt_correct_more10
            FROM default.peas
            GROUP BY st_id
            HAVING cnt_correct_more10 > 10
            ) AS ccm
        LEFT JOIN default.studs AS s ON ccm.st_id=s.st_id) AS a
        GROUP BY test_grp
        ) AS q
    LEFT JOIN (
        SELECT
            test_grp,
            AVG(money) AS avg_money
        FROM (
            SELECT 
                f.st_id,
                f.money,
                s.test_grp
            FROM default.final_project_check AS f
            LEFT JOIN default.studs AS s on f.st_id=s.st_id)
            GROUP BY test_grp
            ) AS n on n.test_grp=q.test_grp
GROUP BY test_grp),

cr_buy as (SELECT
    r.test_grp,
    round(SUM(buyers/all_st), 4) AS CR
FROM (
    SELECT
        s.test_grp,
        uniqExact(f.st_id) AS buyers
    FROM default.final_project_check AS f
    LEFT JOIN default.studs AS s ON f.st_id=s.st_id
    GROUP BY test_grp
    ) AS r
LEFT JOIN (
    SELECT
        s.test_grp,
        uniqExact(p.st_id) AS all_st
    FROM default.peas AS p
    LEFT JOIN default.studs AS s ON p.st_id=s.st_id
    GROUP BY test_grp
    ) AS w ON r.test_grp=w.test_grp
GROUP BY test_grp),

cr_active as (SELECT
    test_grp,
    round((y.buyers/t.st_id), 4) AS CR_active
FROM (
    SELECT
        test_grp,
        COUNT(st_id) AS st_id
    FROM (
        SELECT
            ccm.st_id,
            ccm.cnt_correct_more10,
            s.test_grp
        FROM (
            SELECT
                st_id,
                sum(correct) AS cnt_correct_more10
            FROM default.peas
            GROUP BY st_id
            HAVING cnt_correct_more10 > 10
            ) AS ccm
        LEFT JOIN default.studs AS s ON ccm.st_id=s.st_id) AS a
        group by test_grp) as t
    LEFT JOIN (
        SELECT
            s.test_grp,
            uniqExact(f.st_id) AS buyers
        FROM default.final_project_check AS f
        LEFT JOIN default.studs AS s ON f.st_id=s.st_id
        GROUP BY test_grp
        ) AS y ON t.test_grp=y.test_grp),

cr_math as (SELECT
    test_grp,
    round((cnt_buy_math/cnt_math), 4) AS CR_math
FROM (
    SELECT
        s.test_grp,
        count(u.st_id) AS cnt_math
    FROM (
        SELECT 
            st_id,
            SUM(correct) AS cnt_correct_2_and_more
        FROM default.peas
        WHERE subject = 'Math'
        GROUP BY st_id
        HAVING cnt_correct_2_and_more >= 2
        ) AS u 
    LEFT JOIN default.studs AS s ON u.st_id=s.st_id
    GROUP BY test_grp
    ) AS i
LEFT JOIN (
    SELECT
        test_grp,
        uniqExact(st_id) AS cnt_buy_math
    FROM (
        SELECT *
        FROM default.final_project_check AS f
        LEFT JOIN default.studs AS s ON f.st_id=s.st_id
        WHERE subject = 'Math'
        )
    GROUP BY test_grp) AS l ON l.test_grp=i.test_grp)
    
    SELECT
        a.test_grp AS "group",
        a.ARPU AS "ARPU",
        b.ARPAU AS "ARPAU",
        c.CR AS "CR",
        e.CR_active AS "CR_active",
        d.CR_math AS "CR_math"
    FROM arpu AS a 
    LEFT JOIN arpau AS b ON a.test_grp = b.test_grp
    LEFT JOIN cr_buy AS c ON a.test_grp = c.test_grp
    LEFT JOIN cr_math AS d ON a.test_grp = d.test_grp
    LEFT JOIN cr_active AS e ON a.test_grp = e.test_grp
'''
ph.read_clickhouse(total, connection=connection_default)

,group,ARPU,ARPAU,CR,CR_active,CR_math
0,control,8393.94,605.86,0.0909,0.1181,0.1633
1,pilot,23741.26,822.43,0.2238,0.3333,0.3810
